In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

root = '/kaggle/input/garanti-bbva-data-camp/'
target = 'moved_after_2019'
idx = 'user_id'

In [ ]:
df_train = pd.read_csv(os.path.join(root, 'train_users.csv'))
df_test  = pd.read_csv(os.path.join(root, 'test_users.csv'))
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_edu   = pd.read_csv(os.path.join(root, 'education.csv'))
df_skill = pd.read_csv(os.path.join(root, 'skills.csv'))
df_exp   = pd.read_csv(os.path.join(root, 'work_experiences.csv'))

In [ ]:
df_train = df_train.set_index(idx)
df_test = df_test.set_index(idx)
df_subm = df_subm.set_index(idx)

df_exp = df_exp[df_exp['start_year_month'] < 201901] # see https://www.kaggle.com/competitions/garanti-bbva-data-camp/discussion/383774

### Education Table Preprocess

In [ ]:
df_edu = df_edu[df_edu['school_name'].notnull() & df_edu['degree'].notnull()]

In [ ]:
df_edu

In [ ]:
df_edu

In [ ]:
keywords_dict = {
    'Önlisans': 'Ön Lisans',
    'Ön Lisans': 'Ön Lisans',
    'Ön Lisans': 'Ön Lisans',
    'Associate of Science - AS': 'Ön Lisans',
    'Associates Degree': 'Ön Lisans',
    'Associate Degree': 'Ön Lisans',
    'associate': 'Ön Lisans',
    'ASSOCIATE PROGRAM': 'Ön Lisans',
    'LisansDerecesi': 'Lisans',
    'Bachelor of Science - BSc': 'Lisans',
    'Bachelor of Science - BASc': 'Lisans',
    'Bachelor of Science - B.Sc.': 'Lisans',
    'Bachelor of Science (B.Sc.)': 'Lisans',
    'Bachelor of Business Administration - BBA': 'Lisans',
    'Öğrenci': 'Lisans',
    'Lisans': 'Lisans',
    '1': 'Lisans',
    '2': 'Lisans',
    
    '3': 'Lisans',
    '4': 'Lisans',
    
    '5': 'Lisans',
    'Student': 'Lisans',
    
    '(B.S.)': 'Lisans',
    'BSc ': 'Lisans',
    
    'BA': 'Lisans',
    'Undergraduate': 'Lisans',
    '(BS)': 'Lisans',
    "Engineer's": 'Lisans',
    
    'Bilgisayar': 'Lisans',
    
    'BA': 'Lisans',
    'Arts ': 'Lisans',



    'Master': 'Yuksek Lisans',
    'Yüksek Lisans (Master)': 'Yuksek Lisans',
    'İşletme Yüksek Lisans Programı (MBA)': 'Yuksek Lisans',
    'Master’s': 'Yuksek Lisans',
    
    'MBA': 'Yuksek Lisans',
    'MBA, MIS': 'Yuksek Lisans',
    '(M.Sc.)': 'Yuksek Lisans',
    'MSc': 'Yuksek Lisans',
    "Administration ": 'Yuksek Lisans',
    
    'İşletme': 'Yuksek Lisans',
    'Yüksek lisans ': 'Yuksek Lisans',
    'Yuksek lisans ': 'Yuksek Lisans',
    
    'Business Administration': 'Yuksek Lisans',
    'MS': 'Yuksek Lisans',


    'Doktora': 'Doktora',
    'Philosophy ': 'Doktora',
    'Doctor': 'Doktora',
    '(Ph.D.) ': 'Doktora',
    '(PhD) ': 'Doktora'
}
from fuzzywuzzy import process
def match_string(row, keys_dict):
    degree_str = str(row['degree'])
    match = process.extractOne(degree_str, keys_dict.keys(), score_cutoff=60)
    
    if match:
        return keys_dict.get(match[0])
df_edu['degree'] = df_edu.apply(lambda row: match_string(row, keywords_dict) ,axis=1)

In [ ]:
df_edu

In [ ]:
df_edu

In [ ]:
school_dict = {'ingilizce': 'Yıldız Teknik Üniversitesi',
                 'İstanbul Şehir University': 'İstanbul Şehir Universitesi',
                 'İstanbul Üniversitesi / Istanbul University': 'İstanbul Üniversitesi',
                 'İstanbul University - Cerrahpaşa': 'İstanbul Üniversitesi',
                 'İstanbul University': 'İstanbul Üniversitesi',
                 'istanbul üniversitesi': 'İstanbul Üniversitesi',
                 'İstanbul Teknik Üniversitesi / Istanbul Technical University': 'İstanbul Teknik Üniversitesi',
                 'İstanbul Technical University ': 'İstanbul Teknik Üniversitesi',
                 'İstanbul Beykent Üniversitesi Yönetim Bilşim Sistemleri Y.Lisans (MIS) Tezli': 'İstanbul Beykent Üniversitesi',
                 'İstanbul Bilgi Üniversitesi - Laureate International Universities': 'İstanbul Bilgi Üniversitesi',
                 'albanian': 'Albanca',
                 'İstanbul Aydın University':'İstanbul Aydın Üniversitesi',
                 'İskenderun Teknik Üniversitesi (İSTE) 3.55':'İskenderun Teknik Üniversitesi',
                 'İSTANBUL AYVANSARAY ÜNİVERSİTESİ':'İstanbul Ayvansaray Üniversitesi',
                 'Üsküdar University':'Üsküdar Üniversity',
                 'Özyeğin University':'Özyeğin Üniversitesi',
                 'Ömer Halisdemir University':'Ömer Halisdemir Üniversitesi',
                 'Çankaya University':'Çankaya Üniversitesi',
                 'Yıldız Teknik Üniversitesi Fen Bilimleri Enstitüsü':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University Physics Department':'Yıldız Teknik Üniversitesi',
                 'Yıldız Teknik Üniversitesi / Yıldız Technical University':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University- Graduate School of Natural and Applied Sciences':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University &  Istanbul Institute':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University':'Yıldız Teknik Üniversitesi',
                 'YILDIZ TECHNICAL UNIVERSITY GRADUATE SCHOOL OF SCIENCE AND ENGINEERING':'Yıldız Teknik Üniversitesi',
                 'YILDIZ TECHNICAL UNIVERSITY':'Yıldız Teknik Üniversitesi',
                 'Yıldız Technical University ':'Yıldız Teknik Üniversitesi',
                 'Yıldırım Beyazıt Üniversitesi':'Yıldırım Beyazıt Üniversitesi',
                 'Yildiz Teknik Üniversitesi':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University, Graduate School of Science and Engineering':'Yıldız Teknik Üniversitesi',
                 'Yildiz Technical University ':'Yıldız Teknik Üniversitesi',
                 'Yaşar Üniversitesi (Yaşar University)':'Yaşar Üniversitesi',
                 'Akdeniz University':'Akdeniz Üniversitesi',
                 'Anadolu University':'Anadolu Üniversitesi',
                 'Ankara University, Ankara, Turkey':'Ankara Üniversitesi',
                 'Ankara University, Institue of Nuclear Sciences':'Ankara Üniversitesi',
                 'Ankara Yildirim Beyazit University':'Ankara Yıldırım Beyazıt Üniversitesi',
                 'Ankara Yildirim Beyazit University':'Ankara Yıldırım Beyazıt Üniversitesi',
                 'Ankara Yildirim Beyazit University':'Ankara Yıldırım Beyazıt Üniversitesi',
                 'Atatürk University':'Atatürk Üniversitesi',
                 'Atatürk Üniversitesi Resmi':'Atatürk Üniversitesi',
                 'Ankara Yildirim Beyazit University':'Ankara Yıldırım Beyazıt Üniversitesi',
                 'Ankara University':'Ankara Üniversitesi',
                 'Ankara University':'Ankara Üniversitesi',
                 'Yıldırım Beyazıt University':'Yıldırım Beyazıt Üniversitesi',
                 'Yıldız Teknik Üniversitesi Kimya Mühendisliği Bölümü':'Yıldız Teknik Üniversitesi',
                 'Yıldız Teknik Üniversitesi / Yildiz Technical University':'Yıldız Teknik Üniversitesi',
                 'Yeditepe Üniversitesi / Yeditepe University':'Yeditepe Üniversitesi',
                 'Yeditepe University':'Yeditepe Üniversitesi',
                 'Yaşar University':'Yaşar Üniversitesi',
                 'Yalova University':'Yalova Üniversitesi',
                 'Uludag University':'Uludağ Üniversitesi',
                 'Uludag Universitesi (Lisans)':'Uludağ Üniversitesi',
                 'University of Turkish Aeronautical Association':'Türk Hava Kurumu Üniversitesi',
                 'Trakya Üniversitesi, Fen Bilimleri Enstitüsü':'Trakya Üniversitesi',
                 'Trakya Üniversitesi Teknik Bilimler MYO':'Trakya Üniversitesi',
                 'Trakya Üniversitesi Bilgisayar Mühendisliği':'Trakya Üniversitesi',
                 'Trakya Üniversitesi / Trakya University':'Trakya Üniversitesi',
                 'Trakya Üniversitesi - Computer Technology and Programming':'Trakya Üniversitesi',
                 'Trakya University':'Trakya Üniversitesi',
                 'TED University':'TED Üniversitesi',
                 'TED University / TED Universitesi':'Trakya Üniversitesi',
                 'T.C. Beykent University':'T.C. Beykent Üniversitesi',
                 'Süleyman Demirel Üniversitesi Fenbilimleri Enstitüsü':'Süleyman Demirel Üniversitesi',
                 'Suleyman Demirel University':'Süleyman Demirel Üniversitesi',
                 'Selçuk University':'Selçuk Üniversitesi',
                 'Selcuk University | Turkey':'Selçuk Üniversitesi',
                 'Sakarya Üniversity':'Sakarya Üniversitesi',
                 'Sakarya Üniversitesi/University of Sakarya':'Sakarya Üniversitesi',
                 'Sakarya University, Turkey':'Sakarya Üniversitesi',
                 'Sakarya University, Faculty of Technical Education':'Sakarya Üniversitesi',
                 'Sakarya University, Turkey':'Sakarya Üniversitesi',
                 'Sakarya University':'Sakarya Üniversitesi',
                 'Sabancı Üniversitesi, Istanbul':'Sabancı Üniversitesi',
                 'Sabancı Üniversitesi / Sabanci University':'Sabancı Üniversitesi',
                 'Sabancı Üniversitesi, Istanbul':'Sabancı Üniversitesi',
                 'Orta Doğu Teknik Üniversitesi Kuzey Kıbrıs Kampüsü':'Ortadoğu Teknik Üniversitesi ',
                 'Orta Doğu Teknik Üniversitesi / Middle East Technical University':'Sakarya Üniversitesi',
                 'Okan University':'Okan Üniversitesi',
                 'Nişantaşı University':'Nişantaşı Üniversitesi',
                 'Sakarya University of Applied Sciences':'Sakarya Uygulamalı Bilimler Üniversitesi',
                 'Sakarya Üniversitesi Fen Bilimleri Enstitüsü':'Sakarya Uygulamalı Bilimler Üniversitesi',
                 'Sakarya Üniversitesi Fen Bilimleri Enstitüsü':'Sakarya Uygulamalı Bilimler Üniversitesi',
                 'Sakarya Üniversitesi Fen Bilimleri Enstitüsü':'Sakarya Üniversitesi',
                 'Sakarya Üniversitesi Fen Bilimleri Enstitüsü':'Sakarya Üniversitesi',
                 'Sakarya Üniversitesi Fen Bilimleri Enstitüsü':'Sakarya Üniversitesi',
                 'TOBB University of Economics and Technology':'TOBB Ekonomi ve Teknoloji Üniversitesi',
                 'Yeditepe Üniversitesi / Yeditepe University':'Yeditepe Üniversitesi',
                 
                 'Bahcesehir University':'Bahçeşehir Üniversitesi',
                 'Bahçeşehir University':'Bahçeşehir Üniversitesi',
                 'Balıkesir University':'Balıkesir Üniversitesi',
                 'Baskent University':'Başkent Üniversitesi',
                 'Başkent University':'Başkent Üniversitesi',
                 'Beykent University':'Beykent Üniversitesi',
                 'Bilkent University':'Bilkent Üniversitesi',
                 'Bilkent Üniversitesi / Bilkent University':'Bilkent Üniversitesi',
                 'Bogazici University':'Boğaziçi Üniversitesi',
                 'Bosphorus University':'Boğaziçi Üniversitesi',
                 'Boğaziçi University':'Boğaziçi Üniversitesi',
                 'Boğaziçi Üniversitesi / Bosphorus University':'Boğaziçi Üniversitesi',
                 'Celal Bayar University':'Celal Bayar Üniversitesi',
                 'Cukurova University':'Çukurova Üniversitesi',
                 'Dokuz Eylul University':'Dokuz Eylül Üniversitesi',
                 'Dokuz Eylül Üniversitesi Sosyal Bilimler Enstitüsü':'Dokuz Eylül Üniversitesi',
                 'Dokuz Eylül Üniversitesi Biyotıp ve Genom Enstitüsü':'Dokuz Eylül Üniversitesi',
                 'Dokuz Eylül University, Graduate School of Natural and Applied Sciences':'Dokuz Eylül Üniversitesi',
                 'Dokuz Eylül University Institute of Social Sciences':'Dokuz Eylül Üniversitesi',
                 'Dokuz Eylül University Faculty of Engineering Geophysical Engineering':'Dokuz Eylül Üniversitesi',
                 'Doğuş University':'Doğuş Üniversitesi',
                 'Ege University':'Ege Üniversitesi',
                 'Erciyes University':'Erciyes Üniversitesi',
                 'Eskişehir Technical University':'Eskişehir Teknik Üniversitesi',
                 'Fatih University':'Fatih Üniversitesi',
                 'Firat University':'Fırat Üniversitesi',
                 'Gazi University':'Gazi Üniversitesi',
                 '':'',
                 
                 
                 
                 
                 
                 
                 }



def translate_language(language):
    return school_dict.get(language, language)
df_edu['school_name'] = df_edu['school_name'].apply(translate_language)


df_edu = df_edu.drop_duplicates(['user_id', 'degree'])
df_edu = pd.pivot(df_edu, index='user_id', columns='degree', values='school_name')
df_edu.head()

### Language Table Preprocess

In [ ]:
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_lang = df_lang[df_lang['language'].notnull() & df_lang['proficiency'].notnull()]

In [ ]:
word_dict = {
    'Eski (yaklasik 450-1100)': 'limited_working',
    'Orta (yaklasik 1100-1500)': 'professional_working',
    'Almanca, Orta Yuksek (yaklasik 1050-1500)': 'professional_working',
    '(only very basic speaking skills)': 'elementary',
    '(Temel Duzeyde': 'elementary',
    'Baslangic Seviyesinde ': 'elementary',
    'Eski(842- yaklasik 1400': 'limited_working',
    'Orta(yaklasik 1400-1600)': 'full_professional',
    'Beginner': 'elementary',
    'B2': 'limited_working',
    'Baslangic': 'elementary',
    'Beginner': 'elementary',
}
# Proficiency sütununu doldurma
def fill_proficiency(row):
    proficiency = row['proficiency']
    if pd.isna(proficiency):
        language = row['language']
        choices = []
        for key, value in word_dict.items():
            ratio = process.extractOne(key, language)[1]
            if ratio >= 90:
                choices.append(value)
        if len(choices) > 0:
            return max(set(choices), key=choices.count)
        else:
            return None
    else:
        return proficiency

# Sadece boş proficiency satırlarını doldurma
mask = df_lang['proficiency'].isna()
df_lang.loc[mask, 'proficiency'] = df_lang[mask].apply(fill_proficiency, axis=1)



In [ ]:
lang_rank = {
    'elementary': 1,
    'limited_working': 2,
    'professional_working': 3,
    'full_professional': 4,
    'native_or_bilingual': 5
}

df_lang.loc[:, 'proficiency'] = df_lang.loc[:, 'proficiency'].map(lang_rank)

In [ ]:
language_dict = {'German': 'Almanca', 'german': 'Almanca', 'Almanca': 'Almanca', 'İngilizce': 'İngilizce',
                 'ingilizce': 'İngilizce',
                 'Turkish': 'Türkçe',
                 'turkish': 'Türkçe',
                 'albanian': 'Albanca',
    'arabic': 'Arapça',
    'azerbaijani': 'Azerice',
    'Azerbaycanca': 'Azerice',
    'Azerbaycan Türkçesi': 'Azerice',
    'Azeri': 'Azerice',
    'bulgarian': 'Bulgarca',
    'chinese': 'Çince',
    'deutsch': 'Almanca',
    'ALMANCA': 'Almanca',
    'german': 'Almanca',
    'korean': 'Korece',
    'japonca': 'Japonca',
                 'Albanian': 'Albanca',
    'Arabic': 'Arapça',
    'Azerbaijani': 'Azerice',
    'Bulgarian': 'Bulgarca',
    'Chinese': 'Çince',
    'Deutsch': 'Almanca',
    'German': 'Almanca',
    'Korean': 'Korece',
    'Japonca': 'Japonca',
    'japanese': 'Japonca',
    'Japanese': 'Japonca',
    'latin': 'Latince',
    'persian': 'Farsça',
    'portuguese': 'Portekizce',
    'russian': 'Rusça',
    'English':'İngilizce',
    'english': 'İngilizce',
    'ingilizce':'İngilizce',
    'French':'Fransızca',
    '1- French':'Fransızca',
    'Ingılızce':'İngilizce',
    'Greek':'Yunanca',
    'Ingilizce':'İngilizce',
    'Italian':'İtalyanca',
    'Kurdish':'Kürtçe',
    'Kurdî':'Kürtçe',
    'Kyrgyz':'Kırgız',
    'Lithuanian':'Litvanyaca',
    'Macedonian':'Makedonca',
    'spanish': 'İspanyolca',
    'Norwegian':'Norveççe',
    'Osmanische Sprachen':'Osmanlıca',
    'Ottoman Turkish':'Osmanlıca',
    'Persian':'Farsça',
    'Polish':'Lehçe',
    'Portuguese':'Portekizce',
    'Romanian':'Romanca',
    'Român (Romanian Language)':'Romanca',
    'Russian':'Rusça',
    'Spanish':'İspanyolca',
    'Serbian':'Sırpça',
    'İşaret Dilleri':'İşaret Dili',
    'Sign Languages':'İşaret Dili',
    'Slovenian':'Slovakça',
    'Slavic languages':'Slovakça',
    'Slovene':'Slovakça',
    'Swedish':'İsveççe',
    'TURKISH':'Türkçe',
    '3- Turkish':'Türkçe',
    'Tatar':'Tatarca',
    'Tatar':'Taylandça',
    'Turc':'Türkçe',
    'Turkce':'Türkçe',
    'Turkmen':'Turkmence',
    'Türkmençe':'Turkmence',
    'Turksih':'Türkçe',
    '■ Türkçe ■':'Türkçe',
    'Turkısh':'Türkçe',
    'Türkisch':'Türkçe',
    'Turkish Sign Language':'İşaret Dili',
    'Английский':'İngilizce',
    'Казахский':'Arapça',
    'Крымскотатарский':'Tatarca',
    'Русский':'Rusça',
    'Турецкий':'Rusça',
    'русский':'Rusça',
    'Арабский':'Arapça',
    '■ English ■':'İngilizce',
    '日本の':'Japonca',
    '한국어':'Korece',
    ' English':'İngilizce',
    ' German':'Almanca',
    'English ':'İngilizce',
    'English (Advanced)':'İngilizce',
    'English (B2)':'İngilizce',
    'English (B2, Upper-Intermediate)':'İngilizce',
    'English (Upper-Intermediate)':'İngilizce',
    'English - (YDS : 93,75)':'İngilizce',
    'English - Global Village Sydney Australia':'İngilizce',
    'English - Professional working proficiency':'İngilizce',
    'English C1':'İngilizce',
    'English UK':'İngilizce',
    'İngilizce, Orta (1100-1500)':'İngilizce',
    'English US':'İngilizce',
    'Englisch':'İngilizce',
    'English (US)':'İngilizce',
    'English, Middle (1100-1500)':'İngilizce',
    'English, Old (ca.450-1100)':'İngilizce',
    'English upper intermediate':'İngilizce',
    'English,':'İngilizce',
    'English, Advanced':'İspanyolca',
    'Felemenkçe, Orta (yaklaşık 1050-1350)':'Felemenkçe',
    'Finnish':'Fince',
    'Fransizca':'Fransızca',
    'Fransa':'Fransızca',
    '-English':'İngilizce',
    'Bosnian':'Boşnakça',
    'Dutch':'Almanca',
    '2- English':'İngilizce',
    'Espanol':'İspanyolca',
    'Farsca':'Farsça',
    'Advanced English':'İngilizce',
    
    }





def translate_language(language):
    return language_dict.get(language, language)
df_lang['language'] = df_lang['language'].apply(translate_language)


In [ ]:
df_lang

In [ ]:
df_lang

In [ ]:
df_lang = df_lang.drop_duplicates(['user_id', 'language'])

In [ ]:
df_lang

In [ ]:
df_lang = df_lang.drop_duplicates(['user_id', 'language'])
df_lang = pd.pivot(df_lang, index='user_id', columns='language', values='proficiency')
df_lang = df_lang.fillna(0).astype(int)
df_lang.head(50)

### Skill Table Preprocess

In [ ]:
used_skills = df_skill['skill'].value_counts().iloc[:60].index # we only took most common 60 skills, you can increase it

df_skill = df_skill[df_skill['skill'].isin(used_skills)]
df_skill['experience'] = True

In [ ]:
df_skill = df_skill.drop_duplicates(['user_id', 'skill'])
df_skill = pd.pivot(df_skill, index='user_id', columns='skill', values='experience')
df_skill = df_skill.fillna(0).astype(int)
df_skill.head()

### Experience Table Preprocess

In [ ]:
df_exp = df_exp.sort_values(by=['user_id', 'start_year_month']) # we want nth function to return the last companies by date
df_new = pd.DataFrame()

In [ ]:
locatıon_conv= {
    '(?i).*(istanbul|istan|İSTANBUL|levent|Levent|avcılar|AVCILAR|cıbadem|tunizade|Ataşehir|BEŞİKTAŞ|akırköy|aşakşehir|eylikdüzü|eyoğlu|maslak).*': 'İstanbul',
    '(?i).*(Ankara|ANKARA|nkara|aselsan|kıncı|takule|Bilkent|ilkent|ankiri|Çankırı|eçiören).*': 'Ankara',
    '(?i).*(Adana|dana|ADANA|adana).*': 'Adana',
    '(?i).*(adapazarı|Sakarya|sakarya|akarya|AKARYA).*': 'Sakarya',
    '(?i).*(izmir|aliağa|liağa|alsancak|kordon|İZMİR).*': 'İzmir',
    '(?i).*(Antalya|antalya|talya|NTALYA).*': 'Antalya',
    '(?i).*(Aydin|aydın|YDIN|ydin|ydın).*': 'Aydın',
    '(?i).*(Balıkesir|kesir|balikesir|ALIKESİR|andırma).*': 'Balıkesir',
    '(?i).*(Kocaeli|KOCAEL|izmit|kocaeli).*': 'Kocaeli',
    '(?i).*(manisa|MANİSA|anisa|Manis).*': 'Manisa',
    '(?i).*(ugla|uğla|Muğla|Mugla).*': 'Muğla',
    '(?i).*(Denizli|enizli|DENİZLİ|Denizli).*': 'Denizli',
    '(?i).*(Eskişehir|skişehir|eskisehir|eskısehır|SKİSEHİR|ESKISEHIR).*': 'Eskişehir',
    '(?i).*(Tekirdağ|ekirdağ|TEKİRDAĞ|tekirdag).*': 'Tekirdağ',
    '(?i).*(Elaziğ|elazı|elazi|ELAZİ).*': 'Elazığ',
    '(?i).*(Gaziantep|antep|Gazian|GAZİANTEP).*': 'Gaziantep',
    '(?i).*(Germany|ermany|German|Berlin|lmanya|Almanya|ALMANYA).*': 'Almanya',
    '(?i).*(Amsterdam|hollanda|msterdam).*': 'Hollanda',
    '(?i).*(Amerika|States|Kingdom|United|Birleşik|USA).*': 'Amerika',}
df_exp['location'] = df_exp['location'].replace(locatıon_conv, regex=True)

In [ ]:
df_new['company(1th)'] = df_exp.groupby(idx)['company_id'].nth(-1).astype(str)
df_new['company(2th)'] = df_exp.groupby(idx)['company_id'].nth(-2).astype(str)
df_new['company(3th)'] = df_exp.groupby(idx)['company_id'].nth(-3).astype(str)

df_new['company_location(1th)'] = df_exp.groupby(idx)['location'].nth(-1).astype(str)
df_new['company_location(2th)'] = df_exp.groupby(idx)['location'].nth(-2).astype(str)
df_new['company_location(3th)'] = df_exp.groupby(idx)['location'].nth(-3).astype(str)

df_new['min_exp_time'] = df_exp.groupby(idx)['start_year_month'].min()
df_new['max_exp_time'] = df_exp.groupby(idx)['start_year_month'].max()

df_new['company_count_2018'] = df_exp[df_exp['start_year_month'].gt(201712)].groupby(idx).size()
df_new['company_count_2017'] = df_exp[df_exp['start_year_month'].gt(201612)].groupby(idx).size()
df_new['company_count_2016'] = df_exp[df_exp['start_year_month'].gt(201512)].groupby(idx).size()

In [ ]:
df_exp = df_new
df_exp.head()

### Merge Tables

In [ ]:
df_train[df_edu.columns]   = df_edu[df_edu.columns]
df_train[df_lang.columns]  = df_lang[df_lang.columns]
df_train[df_skill.columns] = df_skill[df_skill.columns]
df_train[df_exp.columns]   = df_exp[df_exp.columns]

df_test[df_edu.columns]   = df_edu[df_edu.columns]
df_test[df_lang.columns]  = df_lang[df_lang.columns]
df_test[df_skill.columns] = df_skill[df_skill.columns]
df_test[df_exp.columns]   = df_exp[df_exp.columns]

In [ ]:
df_train.info()

In [ ]:
df_test.info()

### Encoding

In [ ]:
cat_cols = [col for col in df_test.columns if df_test[col].dtype == 'object']
num_cols = [col for col in df_test.columns if df_test[col].dtype != 'object']

In [ ]:
for col in cat_cols:
    train_cats  = set(df_train[col].unique())
    test_cats   = set(df_test[col].unique())
    common_cats = set.intersection(train_cats, test_cats)
    
    df_train.loc[~df_train[col].isin(common_cats), col] = 'other'
    df_test.loc[~df_test[col].isin(common_cats), col] = 'other'

In [ ]:
df_all = pd.concat([df_train, df_test], axis=0)

for col in cat_cols:
    df_all[col] = df_all[col].factorize()[0]

df_all[cat_cols] = df_all[cat_cols].astype('category')
df_all[num_cols] = df_all[num_cols].fillna(-1)

df_train = df_all.loc[df_train.index, df_train.columns]
df_test = df_all.loc[df_test.index, df_test.columns]

### Training

In [ ]:
X, y = df_train.drop(columns=[target]), df_train[target]
clf = RandomForestClassifier(n_estimators=200, class_weight='balanced_subsample', random_state=42)
cv  = StratifiedKFold(shuffle=True, random_state=42)

scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')

In [ ]:
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

In [ ]:
clf.fit(X, y)
df_subm.loc[df_test.index, target] = clf.predict(df_test)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()

In [ ]:
df_subm.to_csv('submission.csv')